In [3]:
import requests
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import language_tool_python

# Configuration de l'API Ollama
OLLAMA_API_URL = "http://localhost:11434/api/generate"  # Assurez-vous que l'API Ollama est lancée

# Initialiser le modèle d'embedding
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def generate_ideal_response(question):
    """Utilise l'API Ollama pour générer une réponse idéale pour la question donnée."""
    prompt = f"Fournis une réponse claire et concise à la question suivante : {question}"
    
    response = requests.post(
        OLLAMA_API_URL,
        json={"prompt": prompt, "model": "gemma:2b", "stream": False}  # Remplacez 'your_model_name' par le nom de modèle Ollama
    )
    print(response.text)
    
    if response.status_code == 200:
        data = response.json()
        ideal_response = data.get('response', "")
        print("ideal_response.strip() : ", ideal_response.strip())
        return ideal_response.strip()
    else:
        print("Erreur dans la génération de réponse par Ollama :", response.text)
        return None

def calculate_similarity(response_ideal, response_user):
    """Calcule la similarité cosinus entre la réponse idéale et la réponse utilisateur."""
    # Obtenir les embeddings
    embeddings_ideal = embedding_model.encode([response_ideal])
    embeddings_user = embedding_model.encode([response_user])
    
    # Calculer la similarité cosinus
    similarity_score = cosine_similarity(embeddings_ideal, embeddings_user)[0][0]
    return similarity_score

def evaluate_clarity(response_user):
    """Évalue la clarté à l'aide d'une analyse linguistique en français."""
    # Crée une instance de LanguageTool pour le français
    tool = language_tool_python.LanguageTool('fr')
    
    # Vérifie les erreurs dans le texte
    matches = tool.check(response_user)
    
    # Si aucune erreur n'est détectée, la correction sera identique au texte original
    corrected_text = language_tool_python.utils.correct(response_user, matches)
    
    print("Texte corrigé : ", corrected_text)
    
    # Score de clarté basé sur la correction grammaticale et la lisibilité
    return 1 if corrected_text == response_user else 0.5  # Score basique pour clarté

# Question pour laquelle on veut une réponse idéale
question = "Qu'est ce que la photosynthèse ?"
response_user = "C'est le meme processus que les panneaux solaires."
# response_user = "La photosynthèse permet de transformer la lumière, le soleil en énergie pour les plantes."
# response_user = "Processus par lequel les plantes vertes synthétisent des matières organiques grâce à l'énergie lumineuse, en absorbant le gaz carbonique de l'air et en rejetant l'oxygène."

# 1. Générer la réponse idéale
response_ideal = generate_ideal_response(question)

# 2. Évaluer les métriques
if response_ideal:
    relevance_score = calculate_similarity(response_ideal, response_user)
    clarity_score = evaluate_clarity(response_user)
    
    print("Score de pertinence :", relevance_score)
    print("Score de clarté :", clarity_score)

    # Score global moyen
    overall_score = (relevance_score  + clarity_score ) / 2
    print("Score global :", overall_score)
else:
    print("Impossible de générer une réponse idéale.")


{"model":"gemma:2b","created_at":"2024-11-13T09:38:24.7263238Z","response":"La photosynthèse est le processus par lequel les plantes et lesinhardinhard utilisent la lumière du soleil pour convertir le carbone du dioxyde de carbone et l'eau en glucose et l'oxygène.","done":true,"done_reason":"stop","context":[968,2997,235298,559,235298,15508,235313,1645,108,18405,5844,2360,45277,98911,1008,66004,1305,683,2872,80115,865,2510,235303,644,2906,907,683,5831,1686,87695,1654,107,235248,108,235322,2997,235298,559,235298,15508,235313,2516,108,2841,5831,1686,87695,1455,709,46288,755,46078,1437,69236,1008,1437,182646,182646,155032,683,45777,1344,40537,1982,49345,709,105280,1344,13510,4032,495,581,105280,1008,533,235303,7455,659,30859,1008,533,235303,21679,174676,235265],"total_duration":5622170600,"load_duration":70603700,"prompt_eval_count":44,"prompt_eval_duration":134000000,"eval_count":43,"eval_duration":5416000000}
ideal_response.strip() :  La photosynthèse est le processus par lequel les pla

In [23]:
!pip install --upgrade setuptools


Defaulting to user installation because normal site-packages is not writeable
